# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Checkpoint" data-toc-modified-id="Checkpoint-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Checkpoint</a></div><div class="lev1 toc-item"><a href="#Import-Libraries" data-toc-modified-id="Import-Libraries-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Import Libraries</a></div><div class="lev1 toc-item"><a href="#Set-Hyperparameters" data-toc-modified-id="Set-Hyperparameters-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Set Hyperparameters</a></div><div class="lev1 toc-item"><a href="#Build-Graph" data-toc-modified-id="Build-Graph-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Build Graph</a></div><div class="lev1 toc-item"><a href="#Train" data-toc-modified-id="Train-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Train</a></div>

# Checkpoint

In [1]:
import h5py
import pickle

In [2]:
with open('index.pkl', 'rb') as fp:
    word2index, index2word = pickle.load(fp)

In [3]:
with h5py.File('train.h5', 'r') as fh:
    xTrain = fh['xTrain'][:]
    yTrain = fh['yTrain'][:]

# Import Libraries

In [4]:
from keras.layers import Input, Embedding, Dense, Flatten, BatchNormalization, Activation
from keras.models import Model
import keras.backend as K
from keras.callbacks import*
from tqdm import tqdm
import numpy as np

Using TensorFlow backend.


# Set Hyperparameters

In [8]:
VOCAB_SIZE = len(word2index)+1
EMBEDDING_DIM = 256
HIDDEN_SIZE = 512
BATCH_SIZE = 1024
NUM_EPOCHS = 256
STEPS_PER_EPOCH = 2048

# Build Graph

In [9]:
def build():
    """
    Build embedding matrix
    """
    K.clear_session()
    seq = Input(shape=(None,), dtype='int64')
    emb = Embedding(VOCAB_SIZE,
                    EMBEDDING_DIM,
                    mask_zero=False,
                    trainable=True)(seq)
    model = Model(inputs=seq, outputs=emb)
    return model

In [10]:
embedding = build()

In [14]:
inputs = Input(shape=(1,), name='INPUT', dtype='int64')
embSeq = embedding(inputs)
embSeq = Flatten(name='FLATTEN')(embSeq)
dense1 = Dense(HIDDEN_SIZE, activation=None, name='DENSE_1')(embSeq)
bn1 = BatchNormalization(name='BN_1')(dense1)
act1 = Activation('relu', name='RELU_1')(bn1)
dense2 = Dense(HIDDEN_SIZE, activation=None, name='DENSE_2')(act1)
bn2 = BatchNormalization(name='BN_2')(dense2)
act2 = Activation('relu', name='RELU_2')(bn2)
dense3 = Dense(VOCAB_SIZE, activation=None, name='DENSE_3')(act2)
bn3 = BatchNormalization(name='BN_3')(dense3)
out = Activation('softmax', name='OUTPUT')(bn3)
model = Model(inputs=inputs, outputs=out)
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

In [15]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
INPUT (InputLayer)           (None, 1)                 0         
_________________________________________________________________
model_1 (Model)              multiple                  715520    
_________________________________________________________________
FLATTEN (Flatten)            (None, 256)               0         
_________________________________________________________________
DENSE_1 (Dense)              (None, 512)               131584    
_________________________________________________________________
BN_1 (BatchNormalization)    (None, 512)               2048      
_________________________________________________________________
RELU_1 (Activation)          (None, 512)               0         
_________________________________________________________________
DENSE_2 (Dense)              (None, 512)               262656    
__________

# Train

In [16]:
filepath = 'cp_logs/weights.{epoch:03d}-{loss:.6f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
log_string = 'tb_logs/2'
tensorboard = TensorBoard(log_dir=log_string)
callbacks_list = [checkpoint, tensorboard]

In [17]:
def data_generator(data, label, batch_size):
    """
    Yield batches of all data
    """
    count = 0
    while True:
        if count >= len(data): 
            count = 0
        x = np.zeros((batch_size, 1))
        y = np.zeros((batch_size, 1))
        for i in range(batch_size):
            n = i + count
            if n > len(data)-1:
                break
            x[i, :] = data[n]
            y[i, :] = label[n]
        count += batch_size
        yield (x, y)

In [18]:
gen_train = data_generator(xTrain, yTrain, BATCH_SIZE)

In [19]:
history = model.fit_generator(gen_train,
                              steps_per_epoch=STEPS_PER_EPOCH,
                              epochs=NUM_EPOCHS,
                              callbacks=callbacks_list)

Epoch 1/256
2048/2048 [==============================] - 43s - loss: 7.1191    
Epoch 2/256
2048/2048 [==============================] - 46s - loss: 6.6667    
Epoch 3/256
2048/2048 [==============================] - 64s - loss: 6.5900    
Epoch 4/256
2048/2048 [==============================] - 70s - loss: 6.5657    
Epoch 5/256
2048/2048 [==============================] - 86s - loss: 6.5474    
Epoch 6/256
2048/2048 [==============================] - 93s - loss: 6.5328    
Epoch 7/256
2048/2048 [==============================] - 92s - loss: 6.5212    
Epoch 8/256
2048/2048 [==============================] - 100s - loss: 6.5120   
Epoch 9/256
2048/2048 [==============================] - 101s - loss: 6.5096   
Epoch 10/256
2048/2048 [==============================] - 101s - loss: 6.5069   
Epoch 11/256
2048/2048 [==============================] - 102s - loss: 6.5052   
Epoch 12/256
2048/2048 [==============================] - 102s - loss: 6.5021   
Epoch 13/256
2048/2048 [=============

2048/2048 [==============================] - 105s - loss: 6.4773   
Epoch 78/256
2048/2048 [==============================] - 96s - loss: 6.4780    
Epoch 79/256
2048/2048 [==============================] - 96s - loss: 6.4767    
Epoch 80/256
2048/2048 [==============================] - 93s - loss: 6.4775    
Epoch 81/256
2048/2048 [==============================] - 93s - loss: 6.4766    
Epoch 82/256
2048/2048 [==============================] - 89s - loss: 6.4760    
Epoch 83/256
2048/2048 [==============================] - 94s - loss: 6.4756    
Epoch 84/256
2048/2048 [==============================] - 96s - loss: 6.4770    
Epoch 85/256
2048/2048 [==============================] - 96s - loss: 6.4788    
Epoch 86/256
2048/2048 [==============================] - 95s - loss: 6.4749    
Epoch 87/256
2048/2048 [==============================] - 93s - loss: 6.4777    
Epoch 88/256
2048/2048 [==============================] - 96s - loss: 6.4767    
Epoch 89/256
2048/2048 [=================

2048/2048 [==============================] - 102s - loss: 6.4701   
Epoch 160/256
2048/2048 [==============================] - 96s - loss: 6.4707    
Epoch 161/256
2048/2048 [==============================] - 94s - loss: 6.4712    
Epoch 162/256
2048/2048 [==============================] - 92s - loss: 6.4716    
Epoch 163/256
2048/2048 [==============================] - 81s - loss: 6.4700    
Epoch 164/256
2048/2048 [==============================] - 95s - loss: 6.4710    
Epoch 165/256
2048/2048 [==============================] - 86s - loss: 6.4710    
Epoch 166/256
2048/2048 [==============================] - 88s - loss: 6.4697    
Epoch 167/256
2048/2048 [==============================] - 88s - loss: 6.4704    
Epoch 168/256
2048/2048 [==============================] - 90s - loss: 6.4703    
Epoch 169/256
2048/2048 [==============================] - 85s - loss: 6.4713    
Epoch 170/256
2048/2048 [==============================] - 91s - loss: 6.4702    
Epoch 171/256
2048/2048 [=====

2048/2048 [==============================] - 90s - loss: 6.4673    
Epoch 246/256
2048/2048 [==============================] - 85s - loss: 6.4661    
Epoch 247/256
2048/2048 [==============================] - 88s - loss: 6.4660    
Epoch 248/256
2048/2048 [==============================] - 89s - loss: 6.4680    
Epoch 249/256
2048/2048 [==============================] - 82s - loss: 6.4673    
Epoch 250/256
2048/2048 [==============================] - 90s - loss: 6.4672    
Epoch 251/256
2048/2048 [==============================] - 79s - loss: 6.4684    
Epoch 252/256
2048/2048 [==============================] - 86s - loss: 6.4673    
Epoch 253/256
2048/2048 [==============================] - 80s - loss: 6.4678    
Epoch 254/256
2048/2048 [==============================] - 82s - loss: 6.4671    
Epoch 255/256
2048/2048 [==============================] - 89s - loss: 6.4680    
Epoch 256/256
2048/2048 [==============================] - 79s - loss: 6.4674    
